<a href="https://colab.research.google.com/github/isabela-rossetti/analise_exploratoria_preco_gas_pyspark/blob/main/analise_exploratoria_preco_gas_anp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análise Exploratória de dados com PySpark**

O Brasil passa por um aumento significativo de preços, inclusive do GLP - Gás Liquefeito de Petróleo, distribuído para a população em botijões de 13Kg, e que é popularmanete conhecido como Gás de Cozinha ou Botijão de Gás. Nesta análise utilizei os dados resultantes de uma pesquisa conduzida no mês de dezembro de 2021 pela ANP e divulgada no portal de dados abertos do Governo Federal do Brasil. 

## **Preparando o ambiente para utilizar o PySpark**

### **Instalando o Java**

O Apache Spark depende de outros sistemas, portanto, antes do Spark é preciso instalar as dependências. Primeiro, deve-se instalar o java

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

### **Instalando o Apache Spark**

Em seguida, é preciso fazer o download do Spark, e, também, do hadoop, pois o Apache Spark roda sob o HDFS, em sua máquina (no caso aqui, na máquina virtual do Google Colab que você está usando.

In [2]:
# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

### **Configurando o ambiente**

Pronto! Agora precisamos dizer para o sistema onde encontrar o Java e o Spark, que instalamos a pouco neste ambiente.

In [3]:
# Importando a biblioteca os
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

A seguir, vamos precisar da biblioteca findspark que vai nos permitir importar pacotes necessários para o funcionamento do pyspark



In [4]:
# instalando a findspark
!pip install -q findspark

In [5]:
#importando a findspark
import findspark

# iniciando o findspark
findspark.init()

## **Iniciando o PySpark**

Tudo certo até aqui? Ótimo, então agora vamos abrir um sessão Spark para poder trabalhar com este FrameWork.

In [6]:
# importando o pacote necessário
from pyspark.sql import SparkSession

In [7]:
# iniciando o spark context
sc = SparkSession.builder.master('local[*]').getOrCreate()

# Verificando se a sessão foi criada
sc

## **Ingestão dos dados para o Apache Spark**

In [8]:
# download de um conjunto de dados aberto
# fonte: https://dados.gov.br/dataset/serie-historica-de-precos-de-combustiveis-por-revenda
!wget --verbose --show-progress --no-check-certificate https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsan/2021/dados-abertos-precos-2021-12-glp.csv

--2022-12-05 18:34:39--  https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsan/2021/dados-abertos-precos-2021-12-glp.csv
Resolving www.gov.br (www.gov.br)... 161.148.164.31
Connecting to www.gov.br (www.gov.br)|161.148.164.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2623526 (2.5M) [text/csv]
Saving to: ‘dados-abertos-precos-2021-12-glp.csv’

dados-abertos-preco 100%[===================>]   2.50M   465KB/s    in 7.9s    

2022-12-05 18:34:48 (325 KB/s) - ‘dados-abertos-precos-2021-12-glp.csv’ saved [2623526/2623526]



In [9]:
 # carregando o conjunto de dados
df = sc.read.csv("/content/dados-abertos-precos-2021-12-glp.csv", 
                       inferSchema=True, header=True,
                       sep = ';',
                       encoding = "UTF-8")

In [10]:
# Espiando o dataset
df.show()

+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|   Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|     Complemento|           Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|            SE|            SP| SAO VICENTE|CONSIGAZ DISTRIBU...| 01.597.589/0003-81| RUA MANOEL DE ABREU|       790|      QUADRA 108|   CIDADE NAUTICA|11355-400|    GLP|    01/12/2021|         97,99|           null|       R$ / 13 k

## **Conhecendo a estrutura do SparkDataFrame**

In [11]:
# Importações necessárias
from pyspark import *
from pyspark.sql import *

In [12]:
# Verificando o tipo de objeto criado
type(df)

pyspark.sql.dataframe.DataFrame

In [13]:
# Verificando o schema() deste sparkdataframe
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [14]:
# Contando o número de linhas do dataset
df.count()

15765

In [15]:
# Listando as colunas do dataset
df.columns

['Regiao - Sigla',
 'Estado - Sigla',
 'Municipio',
 'Revenda',
 'CNPJ da Revenda',
 'Nome da Rua',
 'Numero Rua',
 'Complemento',
 'Bairro',
 'Cep',
 'Produto',
 'Data da Coleta',
 'Valor de Venda',
 'Valor de Compra',
 'Unidade de Medida',
 'Bandeira']

In [16]:
# Contando o número de colunas do dataset
len(df.columns)

16

In [17]:
# Método para verificar o tipo dos atributos do dataset
df.dtypes

[('Regiao - Sigla', 'string'),
 ('Estado - Sigla', 'string'),
 ('Municipio', 'string'),
 ('Revenda', 'string'),
 ('CNPJ da Revenda', 'string'),
 ('Nome da Rua', 'string'),
 ('Numero Rua', 'string'),
 ('Complemento', 'string'),
 ('Bairro', 'string'),
 ('Cep', 'string'),
 ('Produto', 'string'),
 ('Data da Coleta', 'string'),
 ('Valor de Venda', 'string'),
 ('Valor de Compra', 'string'),
 ('Unidade de Medida', 'string'),
 ('Bandeira', 'string')]

## **Manipulação de colunas do SparkDataFrame**

In [18]:
# importando os métodos com funções para transformações de variáveis
from pyspark.sql.functions import *

In [19]:
# Selecionando somente algumas colunas do dataset
df.select('Revenda','Bandeira','Data da Coleta','Valor de Venda').show(20)

+--------------------+--------------------+--------------+--------------+
|             Revenda|            Bandeira|Data da Coleta|Valor de Venda|
+--------------------+--------------------+--------------+--------------+
|CONSIGAZ DISTRIBU...|            CONSIGAZ|    01/12/2021|         97,99|
|AUTO POSTO AMAPA ...|               FOGAS|    01/12/2021|           115|
|AUTO POSTO ACAUAN...|           AMAZONGÁS|    01/12/2021|           110|
|AUTO POSTO CANIND...|           AMAZONGÁS|    01/12/2021|           115|
|AUTO POSTO AEROPO...|           AMAZONGÁS|    01/12/2021|           115|
|SOUSA & ALVES COM...|SUPERGASBRAS ENERGIA|    01/12/2021|         99,99|
|MARLIM COMBUSTIVE...|            LIQUIGÁS|    01/12/2021|         98,99|
|AUTO SHOPPING DER...|            ULTRAGAZ|    01/12/2021|         98,99|
|LR COMERCIO DE PR...|              BRANCA|    01/12/2021|         99,99|
|CASCOL COMBUSTIVE...|            ULTRAGAZ|    01/12/2021|         99,99|
|CASCOL COMBUSTIVE...|            LIQU

In [20]:
# Outra forma de Selecionar somente algumas colunas do dataset
df.select('Revenda','Bandeira',col('Data da Coleta').alias('Data_Coleta'), col('Valor de Venda').alias('Valor_Venda')).show(20)

+--------------------+--------------------+-----------+-----------+
|             Revenda|            Bandeira|Data_Coleta|Valor_Venda|
+--------------------+--------------------+-----------+-----------+
|CONSIGAZ DISTRIBU...|            CONSIGAZ| 01/12/2021|      97,99|
|AUTO POSTO AMAPA ...|               FOGAS| 01/12/2021|        115|
|AUTO POSTO ACAUAN...|           AMAZONGÁS| 01/12/2021|        110|
|AUTO POSTO CANIND...|           AMAZONGÁS| 01/12/2021|        115|
|AUTO POSTO AEROPO...|           AMAZONGÁS| 01/12/2021|        115|
|SOUSA & ALVES COM...|SUPERGASBRAS ENERGIA| 01/12/2021|      99,99|
|MARLIM COMBUSTIVE...|            LIQUIGÁS| 01/12/2021|      98,99|
|AUTO SHOPPING DER...|            ULTRAGAZ| 01/12/2021|      98,99|
|LR COMERCIO DE PR...|              BRANCA| 01/12/2021|      99,99|
|CASCOL COMBUSTIVE...|            ULTRAGAZ| 01/12/2021|      99,99|
|CASCOL COMBUSTIVE...|            LIQUIGÁS| 01/12/2021|      99,99|
|CASCOL COMBUSTIVE...|            ULTRAGAZ| 01/1

In [21]:
# Outra forma de declarar uma coluna de um dataset
df.select(df.Revenda.alias('Revendedora'),df.Bandeira).show()

+--------------------+--------------------+
|         Revendedora|            Bandeira|
+--------------------+--------------------+
|CONSIGAZ DISTRIBU...|            CONSIGAZ|
|AUTO POSTO AMAPA ...|               FOGAS|
|AUTO POSTO ACAUAN...|           AMAZONGÁS|
|AUTO POSTO CANIND...|           AMAZONGÁS|
|AUTO POSTO AEROPO...|           AMAZONGÁS|
|SOUSA & ALVES COM...|SUPERGASBRAS ENERGIA|
|MARLIM COMBUSTIVE...|            LIQUIGÁS|
|AUTO SHOPPING DER...|            ULTRAGAZ|
|LR COMERCIO DE PR...|              BRANCA|
|CASCOL COMBUSTIVE...|            ULTRAGAZ|
|CASCOL COMBUSTIVE...|            LIQUIGÁS|
|CASCOL COMBUSTIVE...|            ULTRAGAZ|
|POSTO DOS PODERES...|SUPERGASBRAS ENERGIA|
|RIBEIRO DE MENDON...|             COPAGAZ|
|AUTO POSTO MALIBU...|            LIQUIGÁS|
| POSTO ROUXINOL LTDA|             COPAGAZ|
|GABIATTI & GABIAT...|            LIQUIGÁS|
|AUTO POSTO GK EIRELI| NACIONAL GÁS BUTANO|
|GP PETROLEO COMER...|SUPERGASBRAS ENERGIA|
|IRMAOS TEIXEIRA LTDA|          

In [22]:
# Criando uma nova coluna deixando o atributo Bandeira em minúsculo
df.withColumn(colName = "teste", col = lower('Bandeira')).show()

+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+--------------------+
|Regiao - Sigla|Estado - Sigla|   Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|     Complemento|           Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|               teste|
+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+--------------------+
|            SE|            SP| SAO VICENTE|CONSIGAZ DISTRIBU...| 01.597.589/0003-81| RUA MANOEL DE ABREU|       790|      QUADRA 108|   CIDADE NAUTICA|11355-400|    GLP

In [23]:
# Criando uma nova coluna somente replicando o atributo bandeira
df.withColumn("teste", df.Bandeira).show()

+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+--------------------+
|Regiao - Sigla|Estado - Sigla|   Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|     Complemento|           Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|               teste|
+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+--------------------+
|            SE|            SP| SAO VICENTE|CONSIGAZ DISTRIBU...| 01.597.589/0003-81| RUA MANOEL DE ABREU|       790|      QUADRA 108|   CIDADE NAUTICA|11355-400|    GLP

In [24]:
# substituindo caracteres especifico de uma variavel e persistindo o resultado na mesma coluna
df.withColumn(
    colName = 'Data da Coleta', 
    col = regexp_replace('Data da Coleta','/','-')
    ).show()

+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|   Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|     Complemento|           Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|            SE|            SP| SAO VICENTE|CONSIGAZ DISTRIBU...| 01.597.589/0003-81| RUA MANOEL DE ABREU|       790|      QUADRA 108|   CIDADE NAUTICA|11355-400|    GLP|    01-12-2021|         97,99|           null|       R$ / 13 k

In [25]:
# substituindo valores de uma variavel com o metodo replace
df.replace( 
  to_replace = 'GLP',
  value = 'GAS DE COZINHA', 
  subset = 'produto').show()

+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+--------------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|   Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|     Complemento|           Bairro|      Cep|       Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+--------------+--------------+--------------+---------------+-----------------+--------------------+
|            SE|            SP| SAO VICENTE|CONSIGAZ DISTRIBU...| 01.597.589/0003-81| RUA MANOEL DE ABREU|       790|      QUADRA 108|   CIDADE NAUTICA|11355-400|GAS DE COZINHA|    01/12/2021|         97,99|    

In [26]:
# Renomeando colunas
df.withColumnRenamed(existing = 'Estado - Sigla', new = 'Estado').show()

+--------------+------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado|   Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|     Complemento|           Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|            SE|    SP| SAO VICENTE|CONSIGAZ DISTRIBU...| 01.597.589/0003-81| RUA MANOEL DE ABREU|       790|      QUADRA 108|   CIDADE NAUTICA|11355-400|    GLP|    01/12/2021|         97,99|           null|       R$ / 13 kg|            CONSIGAZ|
|       

In [27]:
# Apagando colunas do SparkDataFrame
df.drop('Municipio','Revenda').show()

+--------------+--------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|    CNPJ da Revenda|         Nome da Rua|Numero Rua|     Complemento|           Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|            SE|            SP| 01.597.589/0003-81| RUA MANOEL DE ABREU|       790|      QUADRA 108|   CIDADE NAUTICA|11355-400|    GLP|    01/12/2021|         97,99|           null|       R$ / 13 kg|            CONSIGAZ|
|             N|            AC| 00.529.581/0001-53|    VIA CHICO MENDES|      3570|            null|            

## **Transformações úteis para um Cientista de Dados**

In [28]:
# Verificando os tipos das variáveis
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



### **Corrigindo tipos de variáveis do SparkDataFrame**

In [29]:
# Substituindo '/' por '-' e alterando para a variavel para o formato date
df = df.withColumn(
    colName = 'Data da Coleta', 
    col = to_date(
            col = regexp_replace('Data da Coleta','/','-'),
            format = "dd-MM-yyyy"
          )
    )

# Inspecionando o resultado
df.select('Data da Coleta').printSchema()
df.show()

root
 |-- Data da Coleta: date (nullable = true)

+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|   Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|     Complemento|           Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|            SE|            SP| SAO VICENTE|CONSIGAZ DISTRIBU...| 01.597.589/0003-81| RUA MANOEL DE ABREU|       790|      QUADRA 108|   CIDADE NAUTICA|11355-400|    GLP|    2021-12-

In [30]:
# Transformando o preco em numerica
df = df.withColumn(
    colName = 'Valor de Venda', 
    col = regexp_replace('Valor de Venda',',','.').cast('float')
    )

# Inspecionando o resultado
df.select('Valor de Venda').printSchema()
df.show()


root
 |-- Valor de Venda: float (nullable = true)

+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|   Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|     Complemento|           Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|            SE|            SP| SAO VICENTE|CONSIGAZ DISTRIBU...| 01.597.589/0003-81| RUA MANOEL DE ABREU|       790|      QUADRA 108|   CIDADE NAUTICA|11355-400|    GLP|    2021-12

### **Categorizando variáveis**

In [31]:
# Substituindo '/' por '-' e alterando para a variavel para o formato date
df.withColumn(
    colName = 'Faixa_Preco', 
    col = (
        when(df['Valor de Venda']<100,' abaixo de R$100,00').
        when(((df['Valor de Venda']>=100) & 
            (df['Valor de Venda']<110)),'de R$100 a R$109,99').
       otherwise('R$110,00 ou mais')
    )
    ).show()

+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+-------------------+
|Regiao - Sigla|Estado - Sigla|   Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|     Complemento|           Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|        Faixa_Preco|
+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+-------------------+
|            SE|            SP| SAO VICENTE|CONSIGAZ DISTRIBU...| 01.597.589/0003-81| RUA MANOEL DE ABREU|       790|      QUADRA 108|   CIDADE NAUTICA|11355-400|    GLP|  

### **Criando o atributo Ano de uma variável que contém Data**



In [32]:
# Criando 
df.withColumn(
    colName = 'Ano', 
    col = (df['Data da Coleta'].substr(0, 4)  )
    ).show()

+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+----+
|Regiao - Sigla|Estado - Sigla|   Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|     Complemento|           Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira| Ano|
+--------------+--------------+------------+--------------------+-------------------+--------------------+----------+----------------+-----------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+----+
|            SE|            SP| SAO VICENTE|CONSIGAZ DISTRIBU...| 01.597.589/0003-81| RUA MANOEL DE ABREU|       790|      QUADRA 108|   CIDADE NAUTICA|11355-400|    GLP|    2021-12-01|         97.99|           null| 

## **Filtrando linhas do SparkDataFrame**

In [35]:
# Selecionando somente as cotações realizadas na cidade de Salvador
df.filter(df.Municipio == 'SAO PAULO').show()

+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-----------+--------------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|Complemento|              Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-----------+--------------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|            SE|            SP|SAO PAULO|ELENICE MACHADO F...| 01.390.254/0001-26|   AVENIDA BOTURUSSU|      1118|       null|    PARQUE BOTURUSSU|03802-000|    GLP|    2021-12-02|         105.0|           null|       R$ / 13 kg|SUPERGASBRAS ENERG

In [36]:
# Selecionando somente as cotações realizadas no PARANÁ ou no RIO GRANDE DO SUL
df.filter(
    (df['Estado - Sigla'] == 'PR') | 
    (df['Estado - Sigla'] == 'RS')
    ).show()

+--------------+--------------+-----------------+--------------------+-------------------+--------------------+----------+--------------------+--------------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|        Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|         Complemento|              Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+-----------------+--------------------+-------------------+--------------------+----------+--------------------+--------------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|             S|            RS|SANTA CRUZ DO SUL|ZIMMER, GOETTERT ...| 95.422.259/0001-37|       CORREDOR FREY|       475|                null|            SAO JOAO|96832-010|    GLP|    2021-12-01

In [37]:
# Selecionando somente as cotações no estado de São Paulo no dia 10/12/2021
df.filter(
    (col('Estado - Sigla') == 'SP') & 
    (col('Data da Coleta') == '2021-12-10')
    ).show()

+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-----------+--------------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|Complemento|              Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-----------+--------------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|            SE|            SP| IBITINGA|COMERCIO DE GAS S...| 01.816.591/0001-32|RUA JOAO JOSE CASADO|       101|       null|    DISTR.INDUSTRIAL|14940-000|    GLP|    2021-12-10|         110.0|           null|       R$ / 13 kg|            LIQUIG

In [38]:
# Selecionando somente as cotações no estado da Bahia com preço abaixo de R$80
df.filter(
    (df['Estado - Sigla'] == 'BA') & 
    (df['Valor de Venda'] >120)
    ).show()

+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|Revenda|CNPJ da Revenda|Nome da Rua|Numero Rua|Complemento|Bairro|Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+



In [39]:
# Filtrando linhas com where
df.where(
    (col('Estado - Sigla') == 'BA') & 
    (col('Valor de Venda') < 85)
    ).show()

+--------------+--------------+----------------+--------------------+-------------------+--------------------+----------+------------------+--------------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|Regiao - Sigla|Estado - Sigla|       Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|       Complemento|              Bairro|      Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|            Bandeira|
+--------------+--------------+----------------+--------------------+-------------------+--------------------+----------+------------------+--------------------+---------+-------+--------------+--------------+---------------+-----------------+--------------------+
|            NE|            BA|        SALVADOR|CRISPIM SOUZA COM...| 05.092.369/0001-95|RUA FILADELFO CAR...|       S/N|LOT. JARDIM PIRAJÁ|        ÁGUAS CLARAS|41310-240|    GLP|    2021-12-01|         84

## **Medidas resumos de variáveis numéricas do SparkDataFrame**

In [40]:
# Resumindo variáveis numéricas
df.select('Valor de Venda').describe().show()

+-------+------------------+
|summary|    Valor de Venda|
+-------+------------------+
|  count|             15765|
|   mean|103.29313201371957|
| stddev| 8.951108633678142|
|    min|             77.99|
|    max|             140.0|
+-------+------------------+



In [41]:
# Resumindo variáveis numéricas com summary
df.select('Valor de Venda').summary().show()

+-------+------------------+
|summary|    Valor de Venda|
+-------+------------------+
|  count|             15765|
|   mean|103.29313201371957|
| stddev| 8.951108633678142|
|    min|             77.99|
|    25%|             97.99|
|    50%|             102.0|
|    75%|             110.0|
|    max|             140.0|
+-------+------------------+



In [42]:
# Resumindo variáveis numéricas com summary limitando as estatísticas
df.select('Valor de Venda')\
.summary("min", "5%","25%", "50%","75%", "95%","max")\
.show()

+-------+--------------+
|summary|Valor de Venda|
+-------+--------------+
|    min|         77.99|
|     5%|         89.99|
|    25%|         97.99|
|    50%|         102.0|
|    75%|         110.0|
|    95%|         120.0|
|    max|         140.0|
+-------+--------------+



In [43]:
# Algumas estatísticas individuais
df.select(
  mean('Valor de Venda').alias('Preço médio'),
  percentile_approx('Valor de Venda',0.5).alias('Preço Mediano'),
  percentile_approx('Valor de Venda',0.25).alias('Q1 do Preço'),
  percentile_approx('Valor de Venda',0.75).alias('Q3 do Preço'),
  min('Valor de Venda').alias('Preço Mínimo'),
  max('Valor de Venda').alias('Preço Máximo')
).show()

+------------------+-------------+-----------+-----------+------------+------------+
|       Preço médio|Preço Mediano|Q1 do Preço|Q3 do Preço|Preço Mínimo|Preço Máximo|
+------------------+-------------+-----------+-----------+------------+------------+
|103.29313201371957|        102.0|      97.99|      110.0|       77.99|       140.0|
+------------------+-------------+-----------+-----------+------------+------------+



## **Tabelas de Frequências no PySpark**

In [44]:
# Verificando quantas cotações foram realizadas por região do Brasil
df.groupBy('Regiao - Sigla').count().orderBy(col("count").desc()).show()

+--------------+-----+
|Regiao - Sigla|count|
+--------------+-----+
|            SE| 6905|
|             S| 3037|
|            NE| 2621|
|            CO| 1718|
|             N| 1484|
+--------------+-----+



In [45]:
# Cotações por Estados cruzados por Regioes do Brasil
df.crosstab('Estado - Sigla', 'Regiao - Sigla').show()

+-----------------------------+---+---+---+----+----+
|Estado - Sigla_Regiao - Sigla| CO|  N| NE|   S|  SE|
+-----------------------------+---+---+---+----+----+
|                           MA|  0|  0|144|   0|   0|
|                           GO|663|  0|  0|   0|   0|
|                           CE|  0|  0|573|   0|   0|
|                           AC|  0|114|  0|   0|   0|
|                           PR|  0|  0|  0|1317|   0|
|                           DF|210|  0|  0|   0|   0|
|                           ES|  0|  0|  0|   0| 399|
|                           RJ|  0|  0|  0|   0| 808|
|                           RO|  0|178|  0|   0|   0|
|                           SP|  0|  0|  0|   0|3996|
|                           PB|  0|  0|223|   0|   0|
|                           SE|  0|  0| 89|   0|   0|
|                           RR|  0| 97|  0|   0|   0|
|                           RS|  0|  0|  0|1469|   0|
|                           BA|  0|  0|558|   0|   0|
|                           

## **Análise Estratificada com SparkDataFrame**

In [46]:
# Verificando o preço médio por Região do Brasil
df.groupBy('Regiao - Sigla').mean('Valor de Venda').show()

+--------------+-------------------+
|Regiao - Sigla|avg(Valor de Venda)|
+--------------+-------------------+
|            NE|   99.8615030072564|
|             N| 111.42243928729363|
|             S|  104.3858051268647|
|            SE| 101.12792719040635|
|            CO|  108.2772406821201|
+--------------+-------------------+



In [47]:
# Verificando o preço médio por Região do Brasil
df.groupBy('Estado - Sigla').min('Valor de Venda').show(27)

+--------------+-------------------+
|Estado - Sigla|min(Valor de Venda)|
+--------------+-------------------+
|            SC|              95.99|
|            RO|              100.0|
|            PI|              99.99|
|            AM|               97.0|
|            RR|              105.0|
|            GO|              89.49|
|            TO|              94.99|
|            MT|              84.99|
|            SP|              79.99|
|            PB|              89.99|
|            ES|              79.99|
|            RS|               85.0|
|            MS|               89.0|
|            AL|              86.99|
|            MG|               84.9|
|            PA|               87.0|
|            BA|              77.99|
|            SE|               90.0|
|            PE|              79.99|
|            CE|               88.0|
|            RN|               96.0|
|            RJ|               78.0|
|            MA|               95.0|
|            AC|              110.0|
|